# Tratamento nos dados
Notebook destinado a realizar a preparação dos dados para a otimização

In [41]:
import os
import pandas as pd
import numpy as np

os.chdir('..')
os.chdir('utils')
from funcoes import remover_acentos_e_caracteres

## Lendo o arquivo original

In [42]:
os.chdir('..')
os.chdir('dados')

# Ler o arquivo CSV (substitua 'arquivo.csv' pelo nome do arquivo)
df = pd.read_csv('grade_compilada.csv', sep = ';',)
df.rename(columns={'Período do Dia':'Periodo_dia', 'Parte do Horário':'Horario',
           'Tipo de Curso':'Periodo_Aula', 'Tipo de Turma':'Tipo_Curso',
            'prof':'Professor', 'Observação':'Observacao','Turma':'Curso','Aula':'Materia'}, inplace=True)

# Exclui dados nulos
df.dropna(subset=['Curso'], inplace=True)

#Remove TCCs, que não precisa de sala
df = df[df['Observacao']!='TCC']

# Retirando as meterias sem professor que são diferentes de optativas
# Provavelmente não estão no currículo novo
df = df[~((df['Observacao']!='OPTATIVA') & (df['Professor'].isnull()))]

# Ajusta a coluna campus para conter somente PONTAL E MIRASSOL
df['Campus'] = df['Campus'].replace('PONTAL LABORATÓRIO ', 'PONTAL')

# Ajusta o campus das LCEs, todos os cursos são em mirassol
df.loc[df['Curso'].str.contains('LCE', case=False, na=False), 'Campus'] = 'MIRASOL'

df.tail()


,Campus,Curso,Periodo_dia,Horario,Semestre,Dia,Periodo_Aula,Tipo_Curso,Materia,Professor,Observacao
613,PONTAL,OCEANO,TARDE,2.0,9.0,QUARTA,VESPERTINO,INTEGRAL,INTRODUCAO A MODELAGEM DE PROCESSOS,DOURADO,NaN
614,PONTAL,OCEANO,TARDE,3.0,9.0,QUARTA,VESPERTINO,INTEGRAL,MERGULHO CIENTIFICO(MIRASSOL),BELZ,OPTATIVA
615,PONTAL,OCEANO,TARDE,4.0,9.0,QUARTA,VESPERTINO,INTEGRAL,MERGULHO CIENTIFICO(MIRASSOL),BELZ,OPTATIVA
616,PONTAL,OCEANO,TARDE,1.0,9.0,QUINTA,VESPERTINO,INTEGRAL,INTRODUCAO A MODELAGEM DE PROCESSOS,DOURADO,NaN
617,PONTAL,OCEANO,TARDE,2.0,9.0,QUINTA,VESPERTINO,INTEGRAL,INTRODUCAO A MODELAGEM DE PROCESSOS,DOURADO,NaN


In [43]:
df['Campus'].unique()

array(['MIRASOL', 'PONTAL'], dtype=object)

In [44]:
df[df['Curso'].isin(['LCE', 'LCEFISICA', 'LCEMATEMATICA', 'LCEQUIMICA'])]['Campus'].unique()

array(['MIRASOL'], dtype=object)

In [45]:
df.isna().sum()

Campus            0
Curso             0
Periodo_dia       0
Horario           0
Semestre          0
Dia               0
Periodo_Aula      0
Tipo_Curso        0
Materia           0
Professor         8
Observacao      520
dtype: int64

In [46]:
df[df['Professor'].isnull()]

,Campus,Curso,Periodo_dia,Horario,Semestre,Dia,Periodo_Aula,Tipo_Curso,Materia,Professor,Observacao
395,MIRASOL,LCE-MATEMÁTICA,NOITE,1.0,5.0,SEGUNDA,NOTURNO,NOTURNO,OPTATIVA I TÓPICOS ESPECIAIS I LCE911,NaN,OPTATIVA
396,MIRASOL,LCE-MATEMÁTICA,NOITE,2.0,5.0,SEGUNDA,NOTURNO,NOTURNO,OPTATIVA I TÓPICOS ESPECIAIS I LCE911,NaN,OPTATIVA
421,MIRASOL,LCE-QUÍMICA,NOITE,1.0,5.0,QUARTA,NOTURNO,NOTURNO,OPTATIVA II TÓPICOS ESPECIAIS II LCE912,NaN,OPTATIVA
422,MIRASOL,LCE-QUÍMICA,NOITE,2.0,5.0,QUARTA,NOTURNO,NOTURNO,OPTATIVA II TÓPICOS ESPECIAIS II LCE912,NaN,OPTATIVA
439,MIRASOL,LCE-FÍSICA,NOITE,1.0,7.0,QUARTA,NOTURNO,NOTURNO,OPTATIVA II TÓPICOS ESPECIAIS II LCE912,NaN,OPTATIVA
440,MIRASOL,LCE-FÍSICA,NOITE,2.0,7.0,QUARTA,NOTURNO,NOTURNO,OPTATIVA II TÓPICOS ESPECIAIS II LCE912,NaN,OPTATIVA
473,MIRASOL,LCE-QUÍMICA,NOITE,3.0,7.0,SEGUNDA,NOTURNO,NOTURNO,OPTATIVA I TÓPICOS ESPECIAIS I LCE911,NaN,OPTATIVA
474,MIRASOL,LCE-QUÍMICA,NOITE,4.0,7.0,SEGUNDA,NOTURNO,NOTURNO,OPTATIVA I TÓPICOS ESPECIAIS I LCE911,NaN,OPTATIVA


## Remove acentos e caracteres especiais

In [47]:
colunas_para_tratar = ['Campus', 'Curso', 'Periodo_dia', 'Dia',
       'Periodo_Aula', 'Tipo_Curso', 'Materia', 'Professor', 'Observacao']
df[colunas_para_tratar] = df[colunas_para_tratar].applymap(remover_acentos_e_caracteres)


In [48]:
df.head()

,Campus,Curso,Periodo_dia,Horario,Semestre,Dia,Periodo_Aula,Tipo_Curso,Materia,Professor,Observacao
0,MIRASOL,EAQ,MANHA,1.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUCAO A AQUICULTURA,LAGREZE,NaN
1,MIRASOL,EAQ,MANHA,2.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUCAO A AQUICULTURA,LAGREZE,NaN
2,MIRASOL,EAQ,MANHA,3.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN
3,MIRASOL,EAQ,MANHA,4.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN
4,MIRASOL,EAQ,MANHA,1.0,1.0,TERCA,MATUTINO,INTEGRAL MT,INTRODUCAO A QUALIDADE,SACHSIDA,NaN


## Cria coluna curso+Semestre
Essa coluna é criada porque temos materias diferentes entre os cursos de LCE para os diferentes periodos

In [49]:
df['Semestre_Curso'] = df['Semestre'].astype(int).astype(str) + '_' + df['Curso']
df.head()

,Campus,Curso,Periodo_dia,Horario,Semestre,Dia,Periodo_Aula,Tipo_Curso,Materia,Professor,Observacao,Semestre_Curso
0,MIRASOL,EAQ,MANHA,1.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUCAO A AQUICULTURA,LAGREZE,NaN,1_EAQ
1,MIRASOL,EAQ,MANHA,2.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUCAO A AQUICULTURA,LAGREZE,NaN,1_EAQ
2,MIRASOL,EAQ,MANHA,3.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN,1_EAQ
3,MIRASOL,EAQ,MANHA,4.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN,1_EAQ
4,MIRASOL,EAQ,MANHA,1.0,1.0,TERCA,MATUTINO,INTEGRAL MT,INTRODUCAO A QUALIDADE,SACHSIDA,NaN,1_EAQ


## Quantidade de Hora Aula por materia

In [50]:
hora_aula_materia = df.groupby(['Professor','Materia','Semestre_Curso','Curso'])['Campus'].count().reset_index()
hora_aula_materia.rename(columns={'Campus':'Total_Horas'}, inplace=True)
hora_aula_materia['Periodos_Total'] = np.ceil(hora_aula_materia['Total_Horas']/2)

hora_aula_materia.head()

,Professor,Materia,Semestre_Curso,Curso,Total_Horas,Periodos_Total
0,ALEX,CALCULO DIFERENCIAL E INTEGRAL I,1_EAS,EAS,4,2.0
1,ALEX,CALCULO I,1_ECV,ECV,4,2.0
2,ALEX,GEOMETRIA E CONSTRUCOES CEM319,5_LCEMATEMATICA,LCEMATEMATICA,4,2.0
3,ARMANI,INTRODUCAO A ENGENHARIA AMBIENTAL,1_EAS,EAS,4,2.0
4,ARMANI,POLUICAO DO AR,7_EAS,EAS,3,2.0


In [51]:
len(hora_aula_materia)

213

- Existem materias que temos mais de um professor ofertando
- Fiz o round na quantidade de periodos, pensar se isso não vai interferir, ou se é melhor trabalhar com a quantidade de horas ao invés dos periodos


## Professores e as matérias que ofertam

In [52]:
professores_materias = df[['Professor','Campus','Curso','Tipo_Curso','Materia']].drop_duplicates().reset_index(drop=True)

#remove materias que não tem professor
professores_materias.dropna(subset=['Professor'], inplace=True)
professores_materias.head()

,Professor,Campus,Curso,Tipo_Curso,Materia
0,LAGREZE,MIRASOL,EAQ,INTEGRAL MT,INTRODUCAO A AQUICULTURA
1,CENDON,MIRASOL,EAQ,INTEGRAL MT,PROBABILIDADE
2,SACHSIDA,MIRASOL,EAQ,INTEGRAL MT,INTRODUCAO A QUALIDADE
3,RODOLFO,MIRASOL,EAQ,INTEGRAL MT,INTRODUCAO A QUALIDADE
4,LUCIANA,MIRASOL,EAQ,INTEGRAL MT,GEOMETRIA ANALITICA


In [53]:
professores_materias[professores_materias['Professor'].isnull()]

,Professor,Campus,Curso,Tipo_Curso,Materia


In [54]:
len(professores_materias)

223

## Materias Comuns as licenciaturas

In [55]:
df['Observacao'].value_counts()

COMUM ENTRE LCE                     24
OPTATIVA                            20
MIRASOLPONTAL                        9
SALA 4                               7
PONTALMIRASOL                        4
EAD                                  4
HORARIO DAS 17H30                    2
EAD MANTER                           1
SALA 4 SALA QUE TEM COMPUTADORES     1
Name: Observacao, dtype: int64

In [56]:
materias_comuns = df[df['Observacao']=='COMUM ENTRE LCE'][['Professor','Campus','Materia','Semestre_Curso']]\
    .drop_duplicates().reset_index(drop=True)

materias_comuns

,Professor,Campus,Materia,Semestre_Curso
0,VALDIR,MIRASOL,PRATICA PEDAGOGICA DO ENSINO CEM335,5_LCEFISICA
1,JEINNI,MIRASOL,DIDATICA DAS CIENCIAS CEM334,5_LCEFISICA
2,JEINNI,MIRASOL,ESTAGIO CIENCIAS I CEM336,5_LCEFISICA
3,ELIANE,MIRASOL,DIVULGACAO CIENTIFICA CEM326,5_LCEFISICA
4,VALDIR,MIRASOL,PRATICA PEDAGOGICA DO ENSINO CEM335,5_LCEMATEMATICA
5,JEINNI,MIRASOL,DIDATICA DAS CIENCIAS CEM334,5_LCEMATEMATICA
6,JEINNI,MIRASOL,ESTAGIO CIENCIAS I CEM336,5_LCEMATEMATICA
7,ELIANE,MIRASOL,DIVULGACAO CIENTIFICA CEM326,5_LCEMATEMATICA
8,VALDIR,MIRASOL,PRATICA PEDAGOGICA DO ENSINO CEM335,5_LCEQUIMICA
9,JEINNI,MIRASOL,DIDATICA DAS CIENCIAS CEM334,5_LCEQUIMICA


## Salvando os dataframes

In [57]:
hora_aula_materia.to_csv('hora_aula_materia.csv', index=False)
professores_materias.to_csv('professores_materias.csv', index=False)
materias_comuns.to_csv('materias_comuns.csv', index=False)
df.to_csv('grade_compilada_tratado.csv', index=False)